In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
'''可视化工具 tensorboard使用：
    1.制定面板图上显示的变量
    2.训练过程中将这些变量计算出来输出到文件中
    3.文件解析，./tensorboard --logdir=dir
'''

'''
activation: relu,sigmod,tanh
weight initilizer: he, xavier,normal
optimizer: Adam, Momentum, Gradient Descent
'''

'\nactivation: relu,sigmod,tanh\nweight initilizer: he, xavier,normal\noptimizer: Adam, Momentum, Gradient Descent\n'

In [3]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        #将0-self._num_example的数进行一个混合排列----数据及下标的混合排列
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [ ]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])


  
def convnet(inputs, activation, kernel_initializer):
    # conv1: 神经元图， feature_map, 输出图像
    conv1_1 = tf.layers.conv2d(inputs,
                             32, # output channel number
                             (3,3), # kernel size
                             padding = 'same',
                             activation = activation,
                               #tf.glorot_uniform_initializer
                             kernel_initializer=kernel_initializer,
                             name = 'conv1_1')  

    conv1_2 = tf.layers.conv2d(conv1_1,
                             32, # output channel number
                             (3,3), # kernel size
                             padding = 'same',
                             activation = activation,
                             kernel_initializer=kernel_initializer,
                             name = 'conv1_2')

    '''过卷积层图片大小没有变化，过池化层大小降为1/2'''
    # 16 * 16
    pooling1 = tf.layers.max_pooling2d(conv1_2,
                                       (2, 2), # kernel size
                                       (2, 2), # stride
                                       name = 'pool1')

    conv2_1 = tf.layers.conv2d(pooling1,
                             32, # output channel number
                             (3,3), # kernel size
                             padding = 'same',
                             activation = activation,
                             kernel_initializer=kernel_initializer,
                             name = 'conv2_1')

    conv2_2 = tf.layers.conv2d(conv2_1,
                             32, # output channel number
                             (3,3), # kernel size
                             padding = 'same',
                             activation = activation,
                             kernel_initializer=kernel_initializer,
                             name = 'conv2_2')

    # 8 * 8
    pooling2 = tf.layers.max_pooling2d(conv2_2,
                                       (2, 2), # kernel size
                                       (2, 2), # stride
                                       name = 'pool2')


    conv3_1 = tf.layers.conv2d(pooling2,
                             32, # output channel number
                             (3,3), # kernel size
                             padding = 'same',
                             activation = activation,
                             kernel_initializer=kernel_initializer,
                             name = 'conv3_1')

    conv3_2 = tf.layers.conv2d(conv3_1,
                             32, # output channel number
                             (3,3), # kernel size
                             padding = 'same',
                             activation = activation,
                             kernel_initializer=kernel_initializer,
                             name = 'conv3_2')

    # 4 * 4 * 32
    pooling3 = tf.layers.max_pooling2d(conv3_2,
                                       (2, 2), # kernel size
                                       (2, 2), # stride
                                       name = 'pool3')
    '''------展平------'''
    # [None, 4 * 4 * 32]
    flatten = tf.layers.flatten(pooling3)
    return flatten

#封装实现更改激活函数,初始化方法，
#tf.keras.initializers.he_normal:何凯明初始化方法，在深层网络中才能发挥效用
flatten = convnet(x_image, tf.nn.relu, tf.truncated_normal_initializer(stddev=0.02))#正态分布，保证所有值在一定范围内

y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
'''tensorboard中定义变量需要在网络定义完成之后再定义变量
tensorboard的所有api都在tf.summary下'''

def variable_summary(var,name):
    """ 给一个变量的很多统计量建立一个summary，可以输出到直方图中可以看到，这个函数可以运用到计算中的各个节点上去，tensorboard就能看到各个统计量了"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean',mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.scalar('max',tf.reduce_max(var))
        '''打印直方图'''
        tf.summary.histogram('histogram',var)
         

'''将卷积层的输出分布放到功能函数中看看输出是什么'''
with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(pooling1, 'pooling1')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(pooling2, 'pooling2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
    variable_summary(pooling3, 'pooling3')
    variable_summary(y_, 'y_')
    '''注意，这些都没定义返回值，merge_all（）会自动手集所有调用过tf.summary.scala的节点，而merge则需要指定这就需要返回值了'''


    #'loss':<10,1.1>,<20,1.08>
loss_summary = tf.summary.scalar('loss', loss)

accuracy_summary = tf.summary.scalar('accuracy',accuracy)

#因为输入的是一个图像啊，而且这个图像经过了归一化，这里要反归一化一下
source_image = (x_image+1)*127.5
inputs_summary = tf.summary.image('input_image',source_image)

'''因为有很多变量，如果所有summary都进行sess。run（）的话不合适，太长，tensorflow提供了跟好的api '''
merged_summary = tf.summary.merge_all()#将上面所有summary聚合起来

#制定某些summary来做聚合
merged_summary_test = tf.summary.merge([loss_summary,accuracy_summary,inputs_summary])


'''在tensorboard中指定文件夹不需要自己定义文件名，库会自己做，我们只需要制定文件夹就行了'''
LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR,run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)#tensorboard不会自动创建文件夹自己写程序创建和判断

train_log_dir = os.path.join(run_dir,'train')
test_log_dir = os.path.join(run_dir,'test')
if not os.path.exists(train_log_dir):
    os.makedirs(train_log_dir)
    
if not os.path.exists(test_log_dir):
    os.makedirs(test_log_dir)
    

In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100
output_summary_every_steps = 100

# train 10k: 71.35%
with tf.Session() as sess:
    sess.run(init)
    
    '''tensorboard文件输出部分'''
    train_writer = tf.summary.FileWriter(train_log_dir,sess.graph)#写入指定文件夹，兵指定计算图
    test_write = tf.summary.FileWriter(test_log_dir)#此处并没有指定计算图
   
    #train中每次是调用了nextbatch，数据是无序的，在test中我们希望它是有序的
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        '''tensorboard,想要将summary输出出来，首先要计算出来，加一个变量就行了'''
        eval_ops = [loss,accuracy,train_op]
        should_out_put_summary = ((i+1)%output_summary_every_steps == 0)
        if should_out_put_summary:
            eval_ops.append(merged_summary)
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict={
                x: batch_data,
                y: batch_labels})
        loss_val, acc_val = eval_ops_results[0:2]
        if should_out_put_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str,i+1) #指定步数，随着训练步数的值的变化而变化的点
            test_summary_str = sess.run([merged_summary_test],feed_dict={x: fixed_test_batch_data, y:fixed_test_batch_labels})[0]
            test_write.add_summary(test_summary_str,i+1)
            
   
            
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

### tensorboard打开方式
    tensorboard --logdir=文件夹 名（单个）
     tensorboard --logdir=文件夹1：'自己单独命名1'，文件夹2:‘自己命名2’